In [1]:
import pandas as pd
from sklearn_pandas import DataFrameMapper, cross_val_score
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import sklearn

In [2]:
df = pd.read_csv('train.csv')

def join(r):
    return str(str(r['ID'])+', '+str(r['DATE'])+', '+str(r['TEXT']))

train = pd.DataFrame()

train['text'] = df.apply(lambda r:join(r), 1)

train['label'] = df['truth']

from sklearn.model_selection import train_test_split

tr, tt = train_test_split(train, test_size=0.2)

In [3]:
tr

,text,label
48141,"subject7692, 2017-01-24 18:05:21, hi everyone ...",0
72844,"subject1793, 2016-09-16 15:17:26, bush",0
25513,"subject992, 2018-08-01 06:20:53, crack hell drug",0
104243,"subject1914, 2013-09-11 17:15:08, redacted",0
123215,"subject1885, 2015-03-07 17:51:05, yeah didnt r...",0
...,...,...
110468,"subject3191, 2017-01-23 01:52:50, one horse si...",0
75305,"subject4071, 2018-05-22 06:15:37, spamming cha...",0
28254,"subject5140, 2018-01-29 02:56:07, destroying t...",0
120965,"subject269, 2017-07-28 19:20:02, actually vint...",0


In [ ]:
/media/rishi/Local Disk$

In [3]:
model = ClassificationModel('bert', 'outputs/checkpoint-54000/', args={"num_train_epochs": 10, 
                                                                  "fp16_opt_level" : 'O0',
                                                                 "max_seq_length" : 300,
                                                                  "overwrite_output_dir": True, 
                                                                  "save_eval_checkpoints": False, 
                                                                  "save_model_every_epoch": False})

In [4]:
%load_ext tensorboard
%tensorboard --logdir {'runs/'}

In [7]:
%%time
model.train_model(tr)

/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:246: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Converting to features started. Cache is not used.



Selected optimization level O0:  Pure FP32 training.

Defaults for this optimization level are:
enabled                : True
opt_level              : O0
cast_model_type        : torch.float32
patch_torch_functions  : False
keep_batchnorm_fp32    : None
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O0
cast_model_type        : torch.float32
patch_torch_functions  : False
keep_batchnorm_fp32    : None
master_weights         : False
loss_scale             : 1.0


Running loss: 0.000251


Running loss: 0.009962


Running loss: 0.013505


Running loss: 0.851026


Running loss: 0.015675

RuntimeError: write(): fd 65 failed with No space left on device

In [7]:
result, model_outputs, wrong_predictions = model.eval_model(tt, acc=sklearn.metrics.classification_report)

Converting to features started. Cache is not used.


/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:576: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'mcc': 0.0, 'tp': 0, 'tn': 24237, 'fp': 0, 'fn': 1299, 'acc': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.97     24237\n           1       0.00      0.00      0.00      1299\n\n    accuracy                           0.95     25536\n   macro avg       0.47      0.50      0.49     25536\nweighted avg       0.90      0.95      0.92     25536\n', 'eval_loss': 0.23414792211955054}


/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [16]:
result

{'mcc': 0.0,
 'tp': 0,
 'tn': 24237,
 'fp': 0,
 'fn': 1299,
 'acc': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.97     24237\n           1       0.00      0.00      0.00      1299\n\n    accuracy                           0.95     25536\n   macro avg       0.47      0.50      0.49     25536\nweighted avg       0.90      0.95      0.92     25536\n',
 'eval_loss': 0.23414792211955054}

In [16]:
tt

,text,label
104532,"subject1914, 2015-05-07 21:06:02, end room thi...",0
54715,"subject8973, 2018-04-16 00:55:47, yes",0
81921,"subject203, 2016-09-01 21:28:13, half good alb...",0
39558,"subject2690, 2017-11-25 15:24:31, gt nickname ...",0
110162,"subject9222, 2018-02-20 20:43:27, oh shit ty",0
...,...,...
17080,"subject7355, 2018-02-10 11:39:08, yeah take ye...",0
80693,"subject7263, 2018-06-28 15:55:50, httpsmwikiho...",0
33991,"subject9260, 2017-10-05 22:42:39, copypasting ...",0
40969,"subject2495, 2017-12-03 03:59:17, lebron lefuc...",0
